In [1]:
import numpy as np
import pandas as pd
import os 
import copy
from tqdm import tqdm_notebook
#import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
path = './antai/'
test   = pd.read_csv(path+'Antai_AE_round1_test_20190626.csv')
submission  = pd.read_csv('submission.csv',header = None)

In [3]:
print(test.buyer_admin_id.unique().shape)
print(submission.shape)

(11398,)
(11398, 31)


In [4]:
test.head()

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
0,yy,1061132,189045,2018-07-17 07:43:40,18
1,yy,1081430,10713670,2018-07-17 09:43:15,8
2,yy,1918135,10546322,2018-07-17 11:11:34,9
3,yy,103277,5199603,2018-07-17 13:27:44,24
4,yy,103277,2477273,2018-07-17 13:27:44,22


In [5]:
test = test.sort_values(['buyer_admin_id','irank'])
temp = test.drop_duplicates(subset='buyer_admin_id',keep='first')
temp = temp.reset_index()
temp = temp.drop(columns='index')
temp.head()

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
0,yy,152,8410857,2018-08-22 06:14:24,2
1,yy,282,11721802,2018-08-31 08:06:03,2
2,yy,321,1461800,2018-08-29 03:35:55,2
3,yy,809,2347616,2018-08-06 14:13:29,2
4,yy,870,11382694,2018-08-14 02:12:40,2


In [6]:
temp = temp[['buyer_admin_id','item_id']]
temp.head(10)

,buyer_admin_id,item_id
0,152,8410857
1,282,11721802
2,321,1461800
3,809,2347616
4,870,11382694
5,2410,1991652
6,2956,7116759
7,3866,3738221
8,7277,11977450
9,7944,10039927


# MRR计算

In [7]:
def MRR(test_,submission_):
    if test_.shape[0] != submission_.shape[0]:
        print("size error")
        return 0
    dic = {}
    buyers_score = []
    for item in submission_.values:
        dic[item[0]] = item[1:]
#         for i in range(2,31):
#             dic[item[0]].append(item[i])
    for item in test_.values:
        if item[1] in dic[item[0]]:
            index = np.int(np.argwhere(dic[item[0]] == item[1])[0])+1
#             print(index) 
            buyers_score.append(float(1/index))    
        else:
            buyers_score.append(0)
    score = (np.array(buyers_score).sum() /float(test_.shape[0]))
    return score

In [8]:
score = MRR(temp,submission)
print(score)

1.0
